# **Milestone 3: Sources, Forms, and Quantification of Bias and Discrimination in Supervised Learning**
# **PRACTICE NOTEBOOK 1 - Explore given data**


In this part of the course, we will look for bias using a practical example. A  company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they're not selected or 1 if they are. 

There are 4 practice notebooks in total (current one in red):
1. <font color='red'> **Explore given data: label distribution & check for proxies**</font> 
2. Evaluate model bias "manually"
3. Evaluate model bias using existing libraries (aif360)
4. Example code to get confidence intervals for a metric (nothing to do)

Instructions to complete in each parts are in bold. Intermediate results are given so one can continue the exercise. 

This is notebook number 1. 

In this notebook, we:
- Import modules, define some useful functions, and load the data
- Check for nan values
- Explore the demographics of the candidates and the label distribution per group
- Look for proxies in the data
- See if we can build a model that "guess" the gender/ethnicity

## **0 - Import modules, load data and useful functions**

In [ ]:
#imports 
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Only run if running on Google Colab
!pip3 install pickle5
import pickle5 as pickle
!gdown --id 1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R # download data file from public link and place it in content/ folder

     |████████████████████████████████| 132 kB 3.3 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219316 sha256=acdecb8c4508c3e32548a7f1057606ce1884aab847b21078c445c3982fc20a63
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
Downloading...
From: https://drive.google.com/uc?id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R
To: /content/data.pickle
128MB [00:01, 125MB/s]


In [ ]:
'''
Uncomment the code below if you are running this from your local machine
Note: Place data.pickle file in the same folder as the .ipynb notebook (Download link: https://docs.google.com/uc?export=download&id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg)
'''
# with open('data.pickle', 'rb') as handle:
#     raw_data = pickle.load(handle)   
if 'google.colab' in str(get_ipython()): # if running from colab
  with open('/content/data.pickle', 'rb') as handle:
      raw_data = pickle.load(handle)  
raw_data[:5] #display the first 5 candidates data

,Label,Gender,Ethnicity,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0,Female,White,28.021737,4.351153,2.453895,1.637143,-1.746628,-0.483463,0.034170,1.399225,-0.795440,0.417474,0.214564,-0.471581,1.945645,-0.676217,1.213878,0.015701,1.472670,-0.054158,0.106858,-1.073194,-1.071848,-0.249942,0.634626,-0.732358,2.445728,0.784284,0.112329,1.055362,-0.605459,1.259140,-0.287927,0.214142,-0.644585,1.165376,-0.409198,-0.705823,0.091147,...,-0.655416,-0.012623,0.660826,0.258141,0.036875,-0.229340,0.353817,-0.178814,-0.145229,-0.040692,-0.046980,0.311939,-0.348202,0.271357,0.355443,-0.050447,-0.051816,0.083028,0.184139,0.107824,-0.083415,-0.359288,0.156547,-0.588539,-0.025777,-0.172269,0.331421,0.222768,-0.319124,-0.060476,-0.557444,-0.015627,-0.052749,-0.234189,-0.072384,0.090403,0.376761,0.258914,-0.050558,0.014513
1,0,Female,White,29.603342,-3.407193,0.771800,-2.957411,0.599226,-2.805277,0.329414,-2.055339,-1.194446,-0.633159,2.268302,1.159443,0.899266,-0.472739,0.541605,-1.248643,0.046512,1.225688,0.456477,-1.483071,-0.944882,1.483229,0.512809,0.692537,0.178988,-1.609531,-1.985852,-0.469491,-1.156583,0.475535,-0.041015,-0.214832,-0.681641,1.131433,-0.667814,0.267111,-0.112433,...,0.034605,-0.024487,-0.212205,0.440684,-0.065303,0.409878,0.455144,-0.108402,0.027760,-0.015238,0.027453,0.319960,-0.014589,-0.083241,-0.285702,0.047510,-0.144107,0.405289,-0.044139,-0.287215,0.201876,-0.298703,0.347969,0.029646,0.073052,-0.010259,0.023681,0.373202,-0.525402,-0.198727,-0.198440,-0.158843,0.191984,-0.004532,0.229210,-0.173042,-0.072871,0.442939,-0.054423,0.026959
2,1,Female,Hispanic,26.504283,0.642464,2.522944,-2.197094,2.270646,-0.472510,0.532815,-0.266449,-0.131638,1.038315,-0.865827,-0.811267,-0.381401,-0.801701,-0.485021,0.656005,2.489571,-0.714447,0.658228,-0.075957,-1.159888,-2.334786,-0.253364,-2.073697,-0.939994,-1.177166,0.551689,-1.313316,-0.486217,0.732130,-0.320456,-1.143053,1.297522,-0.617038,0.340978,0.978603,0.398515,...,0.510825,-0.616479,0.644675,0.505319,-0.299894,-0.058435,0.095024,-0.101136,0.042583,0.061005,0.304137,0.259210,-0.022425,0.138097,-0.442536,-0.108350,0.369865,0.151049,0.096285,0.013651,0.175281,0.144344,-0.006250,0.100850,-0.051642,0.122977,-0.088661,-0.229844,-0.272144,0.012633,0.423352,-0.033844,-0.125387,-0.483924,-0.116553,-0.113281,0.015519,0.017111,-0.012309,0.264572
3,0,Female,Hispanic,25.012088,0.895121,-2.092517,3.687830,0.539642,1.988930,1.121646,2.255337,-0.128801,1.148379,1.616247,-2.599757,-0.322807,2.102508,-0.204551,0.069818,0.745222,-0.859875,-2.235995,-0.207436,-1.678697,-0.569024,-0.723122,-0.144833,-1.537487,1.678429,0.501249,-0.230747,0.746559,-0.069959,-0.346651,0.448291,0.283592,-0.445759,-0.529080,0.287333,0.466766,...,-0.153845,-0.049137,-0.023112,-0.173509,-0.265363,0.091898,0.016743,-0.092894,-0.009915,-0.031731,0.153983,0.001281,0.123019,-0.035719,-0.045633,-0.103204,0.089567,0.104990,0.337228,-0.018783,-0.215437,0.268139,-0.125425,0.095183,-0.125172,-0.226467,0.371647,-0.023041,-0.093040,0.338300,-0.280392,0.046582,0.116709,0.133876,0.072716,0.124083,0.213735,-0.149901,-0.217130,0.004403
4,1,Male,Hispanic,27.358934,-2.332423,0.154999,-2.623793,1.682456,1.262280,-1.685565,0.489319,-0.043471,-0.372265,1.778535,-1.145419,2.461327,1.396318,-0.911969,-2.228570,1.378633,-1.512325,-0.440331,-0.111163,-0.885884,-0.840501,1.576620,-0.972075,-2.008346,-0.358732,0.896535,0.562193,0.154542,-1.077315,1.902062,1.728109,0.317205,-0.436143,0.226549,-0.502206,-0.157102,...,0.181831,-0.026589,-0.051453,0.261819,-0.048644,-0.099526,-0.026777,0.039836,-0.168277,0.077232,0.193722,0.093298,-0.075132,-0.063202,0.120167,0.039270,0.350429,0.166559,0.130134,-0.181019,-0.193276,0.312204,-0.187331,-0.029194,-0.212277,-0.463872,0.041810,0.041185,-0.182479,-0.182461,-0.019350,-0.093371,0.0034

In [ ]:
def plot_cm(y_true,y_pred,labels = [1,0],display_labels = [1,0], ax = None):
  cm = confusion_matrix(y_true,y_pred,labels = labels)
  if ax is None:
    fig, ax = plt.subplots()
  else:
    fig = ax.figure
  sns.heatmap(cm, annot=True, ax = ax, cmap='viridis',fmt='g')

  ax.set(xticklabels=display_labels,
          yticklabels=display_labels,
          ylabel="True label",
          xlabel="Predicted label")
  return cm
def split_data_from_df(data):
  y = data['Label'].values
  # g = data['Gender'].values
  # e = data['Ethnicity'].values
  X = data[np.arange(500)].values
  filter_col = ['Ethnicity','Gender'] + [col for col in data if str(col).startswith('Ethnicity_')] + [col for col in data if str(col).startswith('Gender_')] 
  dem = data[filter_col].copy()
  return X,y,dem
def encode(df):
  g_enc = LabelEncoder()
  e_enc = LabelEncoder()
  df['Gender'] = g_enc.fit_transform(df['Gender'])
  df['Ethnicity'] = e_enc.fit_transform(df['Ethnicity'])
  return df, g_enc,e_enc
def resample_equal(df,cat):
  df['uid'] = df[cat] + df['Label'].astype(str)
  enc = LabelEncoder()
  df['uid'] = enc.fit_transform(df['uid'])
  # Resample
  uid = df['uid'].values
  res = imblearn.over_sampling.RandomOverSampler(random_state=6)
  df_res,euid = res.fit_resample(df,uid)
  df_res = pd.DataFrame(df_res,columns = df.columns)
  df_res = df_res.sample(frac=1).reset_index(drop=True)
  df_res['Label'] = df_res['Label'].astype(float)
  return df_res

## **1- Data presentation**

Each row in the data represent a candidate. There are 3 components to the data:
- y : the "ground-truth" label in the "Label" column. This is whether the candidate should actually receive a positive or negative rating (repsectively 1 or 0).
- X : the input features in the columns 0 to 499. There are 500 features for each candidates, extracted from resume and video interview.
- dem : the demographic information in the columns "Gender" or "Ethnicity". 

We extract them from a given dataframe using the pre-defined function "split_data_from_df" to be used as demonstrated below.

In [ ]:
data = raw_data.copy()
X,y,dem = split_data_from_df(data)
print(X.shape, y.shape)
display(dem.head(3))

(31771, 500) (31771,)


,Ethnicity,Gender
0,White,Female
1,White,Female
2,Hispanic,Female


In this section we will:
1. Look for nan values
2. Explore the demographics of the dataset and distribution of labels
3. Check for proxies

## **2. Nan values**

**Questions** : 
- **Check for the number of nan values in each column of *data*.** <br>
    *Expected results*: No nan values in the features or labels. 3997 nan values each for columns Gender and Ethnicity.


- **How are these nan values distributed ?** 
Are the people missing gender information the same as the ones missing ethnicity information ? You can use the missingno library to visualise this on the *dem* dataframe ([see demo here](https://www.geeksforgeeks.org/python-visualize-missing-values-nan-values-using-missingno-library/)). Check your assumptions numerically. <br>
    *Expected results*: You should find that for a given candidate, either both the gender or ethnicity is missing, either all of the information is there. We will call the group of candidates with missing information "the nan group".

In [ ]:
data = raw_data.copy() # work with the "data" dataframe (simple copy of the raw dataframe)

## **3. Demographics of the dataset and average ground-truth success rate for each group**

**Questions** :
- **What groups are represented for gender/ethnicity?** Draw histograms of the *Gender* and *Ethnicity* columns in *data*.
- **Exploring the labels in y, what proportions of all the candidates population receives a positive outcome ?**
- **Now let's calculate the proportion of positive outcome (or success rate) for each gender and ethnicity.** 

Notes:
- Because the label is 0 or 1, calculating the proportion of positive outcome is the same as calculating the mean y values. 
- For this section, we will replace the nan values in the *Gender* and *Ethnicity* columns with a string so that our code will treat these candidates as if they were their own group.
- Usual histogram plotting functions ignore nan values. Because we have changed these into strings, the nan group won't be ignored here.

In [ ]:
# replace nan values with 'nan' string
data = raw_data.copy() # work with the "data" dataframe (simple copy of the raw dataframe)
data = data.fillna("nan")

You should find the following success rates per group:


| Group | Success Rate |
| -: | :-: |
**Mean score per gender**
| Female |  0.383265 |
| Male |  0.384994 |
|nan      |   0.308982 |
|**Mean score per ethnicity**
|Asian     |  0.339046|
| Black    |   0.363655|
|Hispanic   | 0.384433|
|White     |  0.410793|
|nan      |   0.308982|


**Questions** :
- **Ignoring the nan group for now, can you guess which group might experience the most bias with a model trained on this data ?**

    Answer :  Asian
  

**Note on the nan group :** 
As seen in the results, the 'nan' group has the lowest success rate. If we continue the fairness analysis by leaving it in, we will see that it is also the group against which there is the strongest bias. Because we have no prior information on how the "nan" group was constituted and who is in it, it is difficult to draw any conclusion on what this means. In an ideal world, there should be no subgroups constituted at random against which there's an observable bias. However, we do not know whether this group is randomly constituted or not. Possible reasons for missing data could be: (i) people opted out of providing their ethnicity or gender, (ii) these features could have been reverse-engineered from a photo or video and there was no video available for these candidates, or (iii) a reason that correlates somehow with less qualified candidates. 

In the remaining of this course, we remove the nan group from the analysis for simplicity. In a real-life context, the reason behind the bias on the nan group should be investigated and mitigating this bias could be needed. 

In [ ]:
# remove all nans
data = data[data['Gender']!= 'nan'].copy()

## **4. Looking for proxy features**

The features do not include explicitely any protected characteristics related to gender or ethnicity. It can de facto be deemed fair with respect to the *Fairness Through Unawareness* definition. However sometimes it is possible to find proxies for protected attributes in the data, meaning that a specific feature might be strongly correlated with a protected characteristic. 

We can check whether a particular feature is highly correlated with ethnicity or gender by calculating the pearson correlation coefficients of each feature with the gender or ethnicity. Let's demonstrate this for gender only. The same can be done for ethnicity as an exercise.

The following code extracts from the data:

- the input features X
- the gender g




In [ ]:
# copy raw data and drop nan values
df = raw_data.copy()
df = df.dropna()
# Extract X and g
X,_,dem = split_data_from_df(df)
g = dem['Gender'].values
enc = LabelEncoder()
g = enc.fit_transform(g)
print("Label for Male is %d. Label for Female is %d"%(enc.transform(['Male'])[0],enc.transform(['Female'])[0]))

Label for Male is 1. Label for Female is 0


**Questions** : 
- **Calculate the correlation between the gender g and all other features.**
  - using scipy.stats.pearsonr get the correlation coefficient between g and each feature. You can for instance use a loop to go through each features and store results in a list or 1D array. 
  - plot a graph with the correlation coefficient on the y axis and features on the X axis. 
- **Is there any feature correlating highly with gender?**

## **5. Can we still predict the protected characteristic using the entire set of features?**

The previous section should show that there is no single feature that is highly correlated with gender in this case. 

However, can we still infer the gender or ethnicity of a participant from the features themselves? We will try to build a model that predicts the gender or ethnicity from the features.

In this section we use a simple RidgeClassifier to try and do this. The following code defines the classifier, and prepare the data for the task. We will try to predict whether the candidate is a Female or Male. 

As an optional exercise, you may also repeat that exercise for Black/White ethnicity. This is interesting as the dataset is imbalanced regarding these groups (many more White candidates than Black), hence the results are not as straightforward as for Male/Female.

### **5.1. Predict Male/Female gender**

In the following code we:
- encode the *Gender* column into integers
- split the data into train and test set. For this section you will need: 
  - X_train , X_test. are the train and test input features 
  - g_train, g_test. contain the encoded gender (0 for Female; 1 for Male) and are used as labels in this section
- initialize the model

In [ ]:
# copy raw data and drop nan values
df = raw_data.copy()
df = df.dropna()
# encode 'Gender' column
enc = LabelEncoder()
df['Gender'] = enc.fit_transform(df['Gender'])
print("Label for Male is %d. Label for Female is %d"%(enc.transform(['Male'])[0],enc.transform(['Female'])[0]))
# split data train/test (from data without nan)
data_train, data_test = train_test_split(df,test_size = 0.3,random_state=41)
# get X,y, gender, ethnicity
X_train,_,dem_train = split_data_from_df(data_train)
g_train = dem_train['Gender']
X_test,_,dem_test = split_data_from_df(data_test)
g_test = dem_test['Gender']
# define model
model = RidgeClassifier(random_state = 42)

Label for Male is 1. Label for Female is 0


**Instructions:**
- **Fit the model on the train set**
- **Compute the accuracy and confusion matrix on the test set.**  You can use *accuracy_score* (sklearn.metrics) and *plot_cm* (defined in 1)
- **Check the precision, recall and f1 score for each class.** You can use *precision_recall_fscore_support* (sklearn.metrics).

You should obtain the following results:

Overall Accuracy: 0.70

| Group | Precision | Recall | f1 |
| --- | --- | --- | --- | 
| Female | 0.70| 0.68 | 0.69 |
| Male |  0.71| 0.73 | 0.72 |

**Questions:**
- **With what accuracy is it possible to predict if the candidate is a Male or Female ? How does that compare with a random guess ?**
- **What does the precision and recall score mean ? Is there any class imbalance affecting the model ?**

    Answer :

Some possible reasons behind the fact that the features enable gender classification are: 
 - There are some video, photo or voice features that enable gender recognition, or any other ensemble of features that do the same.
 - There are some specific proxy variables that are correlated with gender, as for instance  "Maternity leave" or "Employment length". We have seen from section 4 that this was not the case.


### **5.2. (OPTIONAL) Predict Black/White ethnicity**

In the following code, we perform the same steps as for gender:
- only keep *White* and *Black* candidates
- encode the *Ethnicity* column into integers
- split the data into train and test set. For this section you will need: 
  - X_train , X_test. are the train and test input features 
  - e_train, e_test. contain the encoded ethnicity (0 for Black; 1 for White)
- initialize the model

In [ ]:
# copy raw data and drop nan values
df = raw_data.copy()
df = df.dropna()
# remove non Black or non White for the purpose of this exercise
df = data[(data['Ethnicity'] == 'White') | (data['Ethnicity'] == 'Black')].copy()
# encode 'Ethnicity' column
enc = LabelEncoder()
df['Ethnicity'] = enc.fit_transform(df['Ethnicity'])
print("Label for White is %d. Label for Black is %d"%(enc.transform(['White'])[0],enc.transform(['Black'])[0]))
# split data train/test (from data without nan)
data_train, data_test = train_test_split(df,test_size = 0.3,random_state=41)
# get X,y, gender, ethnicity
X_train,_,dem_train = split_data_from_df(data_train)
e_train = dem_train['Ethnicity']
X_test,_,dem_test = split_data_from_df(data_test)
e_test = dem_test['Ethnicity']
# define model
model = RidgeClassifier(random_state = 42)

Label for White is 1. Label for Black is 0


**Instructions.** As in the gender case: 
- **Fit the model on the train set**
- **Compute the accuracy and confusion matrix on the test set.**  You can use *accuracy_score* (sklearn.metrics) and *plot_cm* (defined in 1)
- **Check the precision, recall and f1 score for each class.** You can use *precision_recall_fscore_support* (sklearn.metrics).
- **Count the number of *Black* and *White* candidates in the test dataset.**

You should obtain the following results:

Overall Accuracy: 0.76


| Group | Precision | Recall | f1 |
| --- | --- | --- | --- | 
| Black | 0.68 | 0.49 | 0.57 |
| White |  0.79| 0.89 | 0.84 |


Test set counts: {*White*: 2997, *Black*: 1411}

<center><img src="https://drive.google.com/uc?id=1O_F5NbhbmnbXiySd63iF26IGD6YQZxCk"></center>

You should observe in the confusion matrix that there are significantly less prediction for Black than for White. This would even happen with a perfect model as there is less candidates categorized as *Black* than *White* in the dataset. In these cases, the precision and recall scores help to judge the quality of a model to predict for each class.

Here we can see that the precision score for both class is around the accuracy score, but the recall score is significantly higher for the *White* class. This means that the model does not think many candidates are *Black* (it misses many), but when if classifies one as *Black* it is rather precise. To tackle this, we can resample the data, i.e. randomly oversampling the minority class or undersampling the majority class so there is the same amount of each class. In theory, one should only resample the training data as the outcome for "real-life" test data is not available so we cannot resample it. However for the purpose of this class, we will resample the whole data.

### **5.3. (OPTIONAL) Predict *Black/White* ethnicity after rebalancing the data**

In the following code, we undersample the data and repeat the experiment.

In [ ]:
# copy raw data and drop nan values
df = raw_data.copy()
df = df.dropna()
# get data
df = data[(data['Ethnicity'] == 'White') | (data['Ethnicity'] == 'Black')].copy()
X,_,dem = split_data_from_df(df)
e = dem['Ethnicity'].values
# resample
print('Original dataset counts', Counter(e))
res = RandomUnderSampler(random_state=42)
X,e = res.fit_resample(X,e)
print('Resampled dataset counts', Counter(e))
# encode ethnicity
enc = LabelEncoder()
e = enc.fit_transform(e)
# split data train/test (from data without nan)
X_train, X_test, e_train, e_test = train_test_split(X,e,test_size = 0.3,random_state=41)

Original dataset counts Counter({'White': 9932, 'Black': 4760})
Resampled dataset counts Counter({'Black': 4760, 'White': 4760})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


**Instructions.** Once again: 
- **Fit the model on the train set**
- **Compute the accuracy and confusion matrix on the test set.**  You can use *accuracy_score* (sklearn.metrics) and *plot_cm* (defined in 1)
- **Check the precision, recall and f1 score for each class.** You can use *precision_recall_fscore_support* (sklearn.metrics).

**Questions:**
- **Is the class imbalance still there ? Can one predict the ethnicity of *White* and *Black* candidates using the input features?**